In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [9]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.dense1 = nn.Linear(2,5)
        self.dense2 = nn.Linear(5,5)
        self.dense3 = nn.Linear(5,1)
        
    def forward(self, x):
        x = self.dense1(x)
        x = F.relu(x)
        x = self.dense2(x)
        x = F.relu(x)
        x = self.dense3(x)
        x = self.sigmoid(x)
        return x

In [10]:
criterion = nn.MSELoss()
net = Net()
print(list(net.parameters()))
optimizer = optim.SGD(net.parameters(),lr=0.01)


[Parameter containing:
tensor([[-0.2356, -0.4787],
        [-0.0213,  0.2120],
        [-0.3703, -0.0195],
        [ 0.2966, -0.1641],
        [ 0.0831,  0.5503]], requires_grad=True), Parameter containing:
tensor([-0.6645,  0.7037, -0.1365,  0.5337,  0.4184], requires_grad=True), Parameter containing:
tensor([[-0.3391, -0.2891,  0.1100,  0.3941,  0.4051],
        [-0.1421,  0.2659, -0.1078,  0.1768,  0.3608],
        [-0.1865,  0.3967, -0.3976,  0.1589,  0.2206],
        [-0.1132,  0.0639,  0.0714, -0.0704,  0.1027],
        [-0.1645,  0.3483,  0.1465,  0.0934,  0.0274]], requires_grad=True), Parameter containing:
tensor([ 0.0955, -0.1162, -0.3976, -0.3735, -0.0307], requires_grad=True), Parameter containing:
tensor([[-0.0027, -0.0458,  0.1504, -0.3220, -0.3760]], requires_grad=True), Parameter containing:
tensor([0.4025], requires_grad=True)]


In [11]:
x = net.parameters().__next__()

In [12]:
print("{}".format(x.data))
print("{}".format(x.grad))

tensor([[-0.2356, -0.4787],
        [-0.0213,  0.2120],
        [-0.3703, -0.0195],
        [ 0.2966, -0.1641],
        [ 0.0831,  0.5503]])
None


In [15]:
input = torch.tensor(torch.rand(1,2), requires_grad=True)
print(input)

tensor([[0.5478, 0.6197]], requires_grad=True)


/Users/annacuddeback/LHCResearch/venv/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """Entry point for launching an IPython kernel.
